## Train a model first

In [ ]:
import sys
sys.path.append('..')
sys.path.append('/usr/local/lib/python3.6/dist-packages/')
import tf_rlib
from tf_rlib.runners import ClassificationRunner
FLAGS = tf_rlib.FLAGS

In [ ]:
# tf_rlib.utils.purge_logs()
tf_rlib.utils.set_gpus('0')
tf_rlib.utils.set_logging('WARN')
tf_rlib.utils.set_amp(True)#NOTE
amp_factor = 2 if FLAGS.amp else 1

FLAGS.bs=128*amp_factor
FLAGS.dim=2
FLAGS.out_dim=10
FLAGS.depth=50
FLAGS.bottleneck=True
layers_per_block = 3 if FLAGS.bottleneck else 2
total_blocks=(FLAGS.depth-2)/layers_per_block
FLAGS.model_alpha=total_blocks*8
FLAGS.lr=1e-3*amp_factor
FLAGS.epochs=1

datasets = tf_rlib.datasets.Cifar10().get_data()
runner = ClassificationRunner(*datasets)

runner.fit(FLAGS.epochs, FLAGS.lr)

## Save the trained model

In [ ]:
w_path = './model.h5'
sm_path = './model'
tftrt_path = './model_tftrt'

### Save weights only

In [ ]:
from tf_rlib.utils.deploy_tools import save_weights, load_weights

save_weights(runner.model, w_path)

### Save as SavedModel format

In [ ]:
from tf_rlib.utils.deploy_tools import convert_to_fp32, save_as_SavedModel

if FLAGS.amp:
    print('converting...')
    runner = convert_to_fp32(runner)
save_as_SavedModel(runner.model, sm_path)

### Save as TFTRT model (convert SavedModel to TFTRT)

In [ ]:
from tf_rlib.utils.deploy_tools import convert_to_TFTRT

convert_to_TFTRT(sm_path, tftrt_path,
                 max_workspace_size_bytes=1<<23,
                 precision='FP16',
                 minimum_segment_size=10,
                 is_dynamic_op=True,
                 use_calibration=True,
                 max_batch_size=32)